In [ ]:
# Load packages
# ---
# NaaVRE:
#  cell: {}
# ...
.libPaths("/home/jovyan/R-packages")

if (!require("pacman")) install.packages("pacman")
pacman::p_load("dplyr", "purrr", "stringr", "httr", "jsonlite", "readr", "here", "lubridate", "tidyr", "taxize", "geosphere", "climwin", "ggpubr")

# Set API keys (Don't containerize cell!)
param_dataverse_api_key 
param_knmi_edr_api_key 

In [ ]:
# Retrieve bud burst data
# ---
# NaaVRE:
#  cell:
#   params:
#    - param_dataverse_api_key:
#       type: String
#       default_value: ''
#   outputs:
#    - event_file: String
#    - occurrence_file: String
#    - extendedmeasurementorfact_file: String
#   dependencies:
#    - name: dplyr
#    - name: purrr
#    - name: stringr
#    - name: httr
#    - name: jsonlite
#    - name: lubridate
#    - name: tidyr
#    - name: taxize
# ...

dir.create("/tmp/data")

# Part I: Retrieve data ---------------------------------------------------

# Function to retrieve data from Dataverse
retrieve_dataverse_data <- function(dataset,
                                    version = ":latest",
                                    server = "demo.dataverse.nl",
                                    key = param_dataverse_api_key) {

  # Check if dataset is provided in right format (i.e., starting with "doi:")
  if(!stringr::str_starts(string = dataset, pattern = "doi:")) {

    dataset_doi <- paste0("doi:", stringr::str_remove(string = dataset, pattern = "DOI:|https://doi.org/"))

  } else {

    dataset_doi <- dataset

  }

  # Retrieve ID that belongs to the data set of interest
  dataset_id <- httr::GET(url = paste0("https://", server, "/api/",
                                       "datasets/:persistentId?persistentId=", dataset_doi),
                          httr::add_headers("X-Dataverse-key" = key)) |>
    httr::content(as = "text", encoding = "UTF-8") |>
    jsonlite::fromJSON() |>
    purrr::pluck("data") |>
    purrr::pluck("id")

  # Retrieve list of data files that are part of the data set
  dataset_files <- httr::GET(url = paste0("https://", server, "/api/",
                                          "datasets/", dataset_id, "/",
                                          "versions/", version, "/",
                                          "files"),
                             httr::add_headers("X-Dataverse-key" = key)) |>
    httr::content(as = "text", encoding = "UTF-8") |>
    jsonlite::fromJSON() |>
    purrr::pluck("data") |>
    purrr::pluck("dataFile")

  # Retrieve each data file in list using their unique IDs
  data <- purrr::map(.x = dataset_files$id,
                     .f = ~{

                       httr::GET(url = paste0("https://", server, "/api/",
                                              "access/datafile/", .x),
                                 httr::add_headers("X-Dataverse-key" = key)) |>
                         httr::content(encoding = "UTF-8")

                     }) |>
    purrr::set_names(stringr::str_remove_all(string = dataset_files$filename, "\\..*"))

  # If API is unsuccessful, prompt message to check DOI, version and/or server
  if(purrr::is_empty(data)) {

    stop("Dataverse API failed to fulfill the request. Check whether the provided dataset DOI, version, and/or server are correct.")

  } else {

    return(data)

  }

}

# Retrieve data
dataverse_list <- retrieve_dataverse_data(dataset = "doi:10.80227/test-QMGPSW")

# Store each table as separate R objects for easier use, and exclude README
purrr::walk2(.x = names(dataverse_list)[-1],
             .y = dataverse_list[-1],
             .f = ~{

               assign(.x, .y, envir = globalenv())

             })

# Part II: create event file (= core of DwC-Archive) ----------------------

## 1. Create help file to create event IDs ####

# Assign area names and abbreviations to sampled trees
d_tree <- tbl_tree %>%
  dplyr::left_join(tbl_area %>%
                     dplyr::select("AreaID", "AreaName", "AreaAbr" = "AreaShortName"),
                   by = "AreaID")

# Assign area names to bud burst data
d_budburst <- tbl_budburst %>%
  dplyr::left_join(d_tree %>%
                     dplyr::select("Area" = "AreaName", "TreeID", "AreaAbr"),
                   by = "TreeID")

# Rename Hoge Veluwe to avoid problems with space between words
d_budburst <- d_budburst %>%
  dplyr::mutate("Area" = stringr::str_replace(string = Area, pattern = " ", replacement = "_"))

# Create eventDate and (hierarchical) eventID of:
# level 1 (<area abbreviation><year>; e.g., HV1988)
# level 2 (<level 1 ID>_<day of year>; e.g., HV1988_119)
# level 3 (<level 2 ID>_<TreeID>; e.g., HV1988_119_412)
hierarchical_events <-
  d_budburst %>%
  dplyr::select("Year", "Month", "Day", "BudburstID", "Area", "AreaAbr", "TreeID") %>%
  dplyr::mutate(eventDate = lubridate::make_date(year = Year, month = Month, day = Day),
                DOY = lubridate::yday(eventDate),
                eventID_L1 = paste(AreaAbr, Year, sep = ""),
                eventID_L2 = paste(eventID_L1, DOY, sep = "_"),
                eventID_L3 = paste(eventID_L2, TreeID, sep = "_"))

## 2. Create event file for Level 1: Years ####

# Get all areas sampled within each year
areas_per_year <-
  d_budburst %>%
  dplyr::group_by(Year) %>%
  dplyr::distinct(Area, .keep_all = TRUE) %>%
  dplyr::summarise(location = paste(Area, collapse = ", ")) %>%
  dplyr::ungroup()

# Create associated event file
d_events_level1 <-
  hierarchical_events %>%
  dplyr::select("eventID_L1", "Year") %>%
  dplyr::distinct(eventID_L1, .keep_all = TRUE) %>%
  dplyr::mutate(eventDate = as.character(Year),
                month = NA,
                day = NA,
                samplingProtocol = NA,
                sampleSizeValue = NA,
                sampleSizeUnit = NA,
                parentEventID = NA,
                decimalLatitude = NA,
                decimalLongitude = NA,
                geodeticDatum = NA,
                minimumElevationInMeters = NA,
                maximumElevationInMeters = NA,
                verbatimLocality = areas_per_year$location[match(.$Year, areas_per_year$Year)]) %>%
  dplyr::rename("eventID" = "eventID_L1",
                "year" = "Year")

## 3. Create event file for level 2: Sampling day within each year ####

# Get all the areas that have been sampled on a specific day (in each year)
areas_per_day <-
  hierarchical_events %>%
  dplyr::group_by(eventDate) %>%
  dplyr::distinct(Area, .keep_all = TRUE) %>%
  dplyr::summarise(location = paste(Area, collapse = ", ")) %>%
  dplyr::ungroup()

# Create event file
d_events_level2 <-
  hierarchical_events %>%
  dplyr::select("eventID_L2", "eventID_L1", "eventDate", "Year", "Month", "Day") %>%
  dplyr::distinct(eventID_L2, .keep_all = TRUE) %>%
  dplyr::mutate(samplingProtocol = NA,
                sampleSizeValue = NA,
                sampleSizeUnit = NA,
                decimalLatitude = NA,
                decimalLongitude = NA,
                geodeticDatum = NA,
                minimumElevationInMeters = NA,
                maximumElevationInMeters = NA,
                verbatimLocality = areas_per_day$location[match(.$eventDate, areas_per_day$eventDate)]) %>%
  dplyr::rename("eventID" = "eventID_L2",
                "parentEventID" = "eventID_L1",
                "year" = "Year",
                "month" = "Month",
                "day" = "Day") %>%
  # Convert dates to characters to avoid merging problems later on
  dplyr::mutate(eventDate = as.character(eventDate))


## 4. Create event file for level 3: bud burst observation on each tree within each year ####

# Connect Tree table to bud burst table and h1
d_budburst <- d_budburst %>%
  dplyr::left_join(hierarchical_events %>%
                     dplyr::select("eventID" = "eventID_L3", "BudburstID"),
                   by = "BudburstID")

# Create associated event file
d_events_level3 <-
  hierarchical_events %>%
  dplyr::select("eventID_L3", "eventID_L2", "eventDate", "Year", "Month", "Day", "TreeID") %>%
  dplyr::mutate(samplingProtocol = "https://doi.org/10.1098/rspb.2000.1363",
                sampleSizeValue = 1,
                sampleSizeUnit = "tree",
                decimalLatitude = d_tree$Latitude[match(.$TreeID, d_tree$TreeID)],
                decimalLongitude = d_tree$Longitude[match(.$TreeID, d_tree$TreeID)],
                minimumElevationInMeters = d_tree$Elevation[match(.$TreeID, d_tree$TreeID)],
                maximumElevationInMeters = minimumElevationInMeters,
                verbatimLocality = d_budburst$Area[match(.$eventID_L3, d_budburst$eventID)]) %>%
  dplyr::rename("eventID" = "eventID_L3",
                "parentEventID" = "eventID_L2",
                "year" = "Year",
                "month" = "Month",
                "day" = "Day") %>%
  dplyr::select(!"TreeID")

# Add geodetic Datum only for events where coordinates are given
d_events_level3 <-
  d_events_level3 %>%
  dplyr::mutate(geodeticDatum = dplyr::case_when(!is.na(decimalLatitude) ~ "EPSG:4326",
                                                 TRUE ~ NA_character_),
                eventDate = as.character(eventDate))

# Combine all three event files into the final event-core file
event <-
  dplyr::bind_rows(d_events_level1, d_events_level2, d_events_level3) %>%
  dplyr::arrange(eventDate, eventID)

# Add DwC columns that apply to all event levels
event <-
  event %>%
  dplyr::mutate(language = "en",
                country = "Netherlands",
                countryCode = "NL",
                institutionID = "https://ror.org/01g25jp36",
                institutionCode = "NIOO",
                type = "Event") %>%
  # Reorder event file according to GBIF list
  dplyr::select("eventID", "parentEventID", "samplingProtocol", "sampleSizeValue",
                "sampleSizeUnit", "eventDate", "year", "month", "day", "country",
                "countryCode", "verbatimLocality", "minimumElevationInMeters",
                "maximumElevationInMeters", "decimalLatitude", "decimalLongitude",
                "geodeticDatum", "type", "language", "institutionID", "institutionCode") %>%
  # Rename "Hoge Veluwe" back to original name
  dplyr::mutate(verbatimLocality = stringr::str_replace(string = verbatimLocality, pattern = "_", replacement = " "))

# Save file as text file
event_file <- "/tmp/data/event.csv"
write.csv(event, file = event_file, row.names = FALSE)


# Part III. Create occurrence table ---------------------------------------

# Merge tables to assign tree species to each measurement
tree_species <-
  d_tree %>%
  dplyr::select("TreeID", "TreeSpeciesID") %>%
  #dplyr::select("TreeID", "TreeSpeciesID", "Remarks") %>%
  dplyr::left_join(tbl_treeSpecies, by = "TreeSpeciesID") %>%
  dplyr::right_join(d_budburst, by = "TreeID")


## 1. Get the taxonomic information of all species ####

# Add scientific names to tree table
tree_species <-
  tree_species %>%
  dplyr::mutate(species = dplyr::case_when(TreeSpeciesName == "European oak" ~ "Quercus robur",
                                           TreeSpeciesName == "American oak" ~ "Quercus rubra",
                                           TreeSpeciesName == "Larch" ~ "Larix kaempferi",
                                           TreeSpeciesName == "Pine" ~ "Pinus sylvestris",
                                           TreeSpeciesName == "Birch" ~ "Betula pendula",
                                           TRUE ~ "Tracheophyta"))

# Get all scientific Names to query the taxonomic information in the next step
sciNames <- unique(tree_species$species)

# Query for all species
tax <- taxize::get_gbifid_(sci = sciNames) %>%
  dplyr::bind_rows() %>%
  dplyr::filter(status == "ACCEPTED" & matchtype == "EXACT") %>%
  tidyr::separate(canonicalname, c("Genus", "specificEpithet"), remove = FALSE) %>%
  dplyr::select("scientificName" = "scientificname", "canonicalname",
                "kingdom", "phylum", "class", "order", "family", "genus", "specificEpithet")


# Bind taxonomic information to each observation
tree_species_tax <- dplyr::left_join(tree_species,
                                     tax,
                                     by = c("species" = "canonicalname"))


## 2. Create occurrence IDs ####

# Check whether there is any occasion in which more than one tree was sampled at a sampling event
# (should not be the case here as we know that one measurement is only one tree at a time)
if(d_budburst %>% dplyr::count(eventID) %>% dplyr::filter(n > 1) %>% nrow() > 0) {

  stop(paste("In", d_budburst %>% dplyr::count(eventID) %>% dplyr::filter(n > 1) %>% nrow(),
             "instances of an event, more than one tree was sampled.",
             "This should not be the case for level-3 events."))

}

# Create occurrenceID by extending eventID with '_1'
occID <-
  d_events_level3 %>%
  dplyr::arrange(eventDate) %>%
  dplyr::mutate(occurrenceID = paste(eventID, 1, sep = "_"))

# Create occurrence file
occurrence <-
  tree_species_tax %>%
  dplyr::select("eventID", "Observer",
                "kingdom", "phylum", "class", "order",
                "family", "genus", "specificEpithet", "scientificName", "TreeID") %>%
  dplyr::mutate(individualCount = 1,
                basisOfRecord = "HumanObservation",
                occurrenceStatus = "present",
                occurrenceRemarks = NA,
                recordedByID = Observer,
                occurrenceID = occID$occurrenceID[match(.$eventID, occID$eventID)]) %>%
  dplyr::rename("organismID" = "TreeID") %>%
  dplyr::select("eventID", "occurrenceID", "recordedByID",
                "individualCount", "basisOfRecord", "occurrenceStatus",
                "occurrenceRemarks", "organismID", "scientificName", "kingdom", "phylum", "class", "order",
                "family", "genus", "specificEpithet")

# Save file as text file
occurrence_file <- "/tmp/data/occurrence.csv"
write.csv(occurrence, file = occurrence_file, row.names = FALSE)

# Part IV: Create Measurement or fact file --------------------------------

## 1. Create measurement or fact file ####
measurement_or_fact <-
  tree_species_tax %>%
  tidyr::pivot_longer(col = c("TreeTopScore", "TreeAllScore"),
                      names_to = "measurementType",
                      values_to = "measurementValue")  %>%
  dplyr::select("eventID", "measurementType", "measurementValue")%>%
  dplyr::mutate(measurementUnit = NA,
                measurementMethod = "https://doi.org/10.1098/rspb.2000.1363",
                measurementRemarks = NA)

## 2. Create measurementID ####

# Number the measurements per occurrence
measurement_or_fact <-
  measurement_or_fact %>%
  dplyr::group_by(eventID) %>%
  dplyr::mutate("ID" = 1:dplyr::n()) %>%
  dplyr::ungroup()

# Add occurrenceID & create measurementID by extending occurrenceID by number of measurement
measurement_or_fact <-
  measurement_or_fact %>%
  dplyr::left_join(occurrence %>%
                     dplyr::select("occurrenceID", "eventID"),
                   by = "eventID") %>%
  dplyr::mutate(measurementID = paste(occurrenceID, ID, sep = "_")) %>%
  dplyr::select(!c("ID", "occurrenceID")) %>%
  # Rename measurement types to fit more controlled vocabulary
  dplyr::mutate(measurementType = dplyr::case_when(measurementType == "TreeTopScore" ~ "bud burst stage (PO:0025532) of the tree crown",
                                                   measurementType == "TreeAllScore" ~ "bud burst stage (PO:0025532) of the whole tree")) %>%
  # Reorder columns according to GBIF list
  dplyr::select("measurementID", "eventID", "measurementType", "measurementValue",
                "measurementUnit", "measurementMethod", "measurementRemarks")

# Save file as text file
extendedmeasurementorfact_file <- "/tmp/data/extendedmeasurementorfact.csv"
write.csv(measurement_or_fact, file = extendedmeasurementorfact_file, row.names = FALSE)

In [ ]:
# Process bud burst data
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - event_file: String
#    - occurrence_file: String
#    - extendedmeasurementorfact_file: String
#   outputs:
#    - budburst_file: String
#   dependencies:
#    - name: dplyr
#    - name: stringr
#    - name: lubridate
# ...


# Part I: Preparation  ----------------------------------------------------

# Load DwC-A files
event <- read.csv(event_file)
occ <- read.csv(occurrence_file)
mof <- read.csv(extendedmeasurementorfact_file)

# Connect measurements with events (i.e., trees)
d_bb <- dplyr::left_join(event, mof, by = "eventID")

# Get organism ID for each event
d_bb <-
  occ %>%
  dplyr::select("eventID", "organismID", "scientificName") %>%
  dplyr::right_join(d_bb, by = "eventID", relationship = "many-to-many")

# Part II: Queries --------------------------------------------------------

# 1. Define bud burst criteria ####

# Set measurement type criterion: TopScore (= bud burst stage (PO:0025532) of the tree crown)
crit_measType <- "bud burst stage (PO:0025532) of the tree crown"

# First day where measurement value (of the tree crown) > 1 is bud burst date
crit_measValue <- 1


# 2. Select dates before reaching criterion and when reaching criterion per tree per year ####

# Filter for tree crown, which is used to determine bud burst dates
d_bb_crown <-
  d_bb %>%
  dplyr::filter(measurementType == crit_measType) %>%
  ## Convert date to day of year for easier calculations
  dplyr::mutate(DOY = lubridate::yday(eventDate))

# Bud burst date is the first date where the bud burst stage of the tree crown is scored >= 1
# First, we get the earliest dates (and associated bud burst stage values) per tree per year that matches this criterion
min_bb <- d_bb_crown %>%
  dplyr::filter(measurementValue >= crit_measValue) %>%
  dplyr::summarise(min_DOY_above_criterion = min(DOY),
                   min_value = measurementValue[DOY == min_DOY_above_criterion],
                   .by = c("year", "organismID"))

# Join earliest date and value back into main data frame
d_bb_crown2 <- dplyr::left_join(d_bb_crown,
                                min_bb,
                                by = c("year", "organismID"))

# Second, we also need the latest date (and associated value) at which the bud burst stage was below the criterion
# This is used to interpolate a bud burst date in cases where the tree reached stage 1 in-between two field visits
# In other cases, where stage 1 was observed during a field visit, the date of that visit is used as the bud burst date
max_bb <- d_bb_crown2 %>%
  dplyr::filter(measurementValue < crit_measValue & DOY < min_DOY_above_criterion,
                .by = c("year", "organismID")) %>%
  dplyr::summarise(max_DOY_below_criterion = max(DOY),
                   max_value = measurementValue[DOY == max_DOY_below_criterion],
                   .by = c("year", "organismID"))

# Join earliest date at criterion and latest date below criterion, and their recorded bud burst values
min_max_bb <- dplyr::left_join(min_bb,
                               max_bb,
                               by = c("year", "organismID"))

# 3. Calculate bud burst dates per tree per year ####

# For trees for which a stage of 1 was observed during a visit, use the date of that visit as the bud burst date
match_criterion <- min_max_bb %>%
  dplyr::filter(min_value == crit_measValue) %>%
  dplyr::mutate(bud_burst_date = min_DOY_above_criterion + lubridate::make_date(year, 1, 1) - 1) %>%
  dplyr::rename("bud_burst_DOY" = "min_DOY_above_criterion")

# For trees for which a stage 1 was NOT observed during a visit, (linearly) interpolate the date
# from the latest date before criterion and the earliest date after reaching criterion
interpolated <- min_max_bb %>%
  dplyr::filter(min_value != crit_measValue) %>%
  dplyr::mutate(diff_date = min_DOY_above_criterion - max_DOY_below_criterion,
                diff_value = min_value - max_value,
                value_per_day = diff_value / diff_date,
                days_to_reach_criterion = (crit_measValue - max_value) / value_per_day,
                interpolated_DOY = max_DOY_below_criterion + days_to_reach_criterion,
                interpolated_value  = max_value + days_to_reach_criterion * value_per_day) %>%
  dplyr::mutate(bud_burst_date = round(interpolated_DOY) + lubridate::make_date(year, 1, 1) - 1) %>%
  dplyr::rename("bud_burst_DOY" = "interpolated_DOY")

# Merge determined or calculated bud burst dates and bud burst DOYs per tree per year together
# and add auxiliary info (i.e., locality, scientific name) back in from main file
bud_burst_dates <- dplyr::bind_rows(match_criterion %>%
                                      dplyr::select("year", "organismID", "bud_burst_date", "bud_burst_DOY"),
                                    interpolated %>%
                                      dplyr::select("year", "organismID", "bud_burst_date", "bud_burst_DOY")) %>%
  dplyr::left_join(d_bb_crown %>%
                     dplyr::select("year", "organismID", "verbatimLocality", "scientificName") %>%
                     dplyr::distinct(),
                   by = c("year", "organismID")) %>%
  dplyr::arrange(year, organismID)

# Save dates as csv
budburst_file <- "/tmp/data/annual_budburst_per_tree.csv"
write.csv(bud_burst_dates, budburst_file, row.names = FALSE)

In [ ]:
# Retrieve KNMI temperature data
# ---
# NaaVRE:
#  cell:
#   params:
#    - param_knmi_edr_api_key:
#       type: String
#       default_value: ''
#   outputs:
#    - temperature_file: String
#   dependencies:
#    - name: dplyr
#    - name: purrr
#    - name: stringr
#    - name: httr
#    - name: jsonlite
#    - name: readr
#    - name: lubridate
#    - name: tidyr
# ...

dir.create("/tmp/data")

# I. Function to retrieve data from the EDR API ------------------------------

# Arguments:
# bbox: spatial bounding box for which to retrieve data. Vector of four numeric values, indicating western-most, southern-most, eastern-most and northern-most point of the bounding box (in decimal degrees).
# variable: weather variable of interest. Either "mean temperature", "max temperature", "min temperature" or "precipitation".
# start_date: start date of period for which to retrieve data. Date format "yyyy-mm-dd". Use e.g., `lubridate::make_date()`.
# start_time: start time of period for which to retrieve data. Character string of format "hh:mm:ss". Default: "00:00:00".
# end_date: end date of period for which to retrieve data. Date format "yyyy-mm-dd". Use e.g., `lubridate::make_date()`.
# end_time: end time of period for which to retrieve data. Character string of format "hh:mm:ss". Default: "23:59:59".
# knmi_edr_key: user-specific KNMI EDR API key (request here: https://developer.dataplatform.knmi.nl/edr-api#token). Character string.

retrieve_knmi_edr_data <- function(bbox,
                                   variable = c("mean temperature", "max temperature",
                                                "min temperature", "precipitation"),
                                   start_date,
                                   start_time = "00:00:00",
                                   end_date,
                                   end_time = "23:59:59",
                                   knmi_edr_key = param_knmi_edr_api_key) {

  # Create KNMI variable lookup table to match variable inputs to KNMI collection & parameter names
  knmi_var_lookup <- tibble::tibble(
    collection = c("Tg1", "Tx1", "Tn1", "Rd1"),
    parameter = c("temperature", "temperature", "temperature", "precipitation"),
    var_name = c("mean temperature", "max temperature", "min temperature", "precipitation")
  )

  if(!(variable %in% knmi_var_lookup$var_name)) {

    stop("The weather variable you provided does not exist. Select one of: 'mean temperature', 'max temperature', 'min temperature', or 'precipitation'.")

  }

  # Check that start_date and end_date are of class 'Date' to ensure that we can retrieve
  if(!any(class(start_date) == "Date", class(end_date) == "Date")) {

    stop("Please provide dates as 'yyyy-mm-dd'.")

  }

  repeat({ # If retrieving data from KNMI EDR API fails, try again

    # Send GET request to KNMI EDR API
    edr_get <- httr::GET(url = paste0("https://api.dataplatform.knmi.nl/edr/collections/",
                                      knmi_var_lookup |> dplyr::filter(var_name == variable) |> dplyr::pull("collection"),
                                      "/cube?bbox=", paste(bbox, collapse = "%2C"),
                                      "&z=0",
                                      "&parameter-name=",
                                      knmi_var_lookup |> dplyr::filter(var_name == variable) |> dplyr::pull("parameter"),
                                      "&datetime=",
                                      start_date, "T",
                                      stringr::str_replace_all(string = start_time,
                                                               pattern = ":",
                                                               replacement = "%3A"), "Z%2F",
                                      end_date, "T",
                                      stringr::str_replace_all(string = end_time,
                                                               pattern = ":",
                                                               replacement = "%3A"), "Z"),
                         httr::add_headers(Authorization = knmi_edr_key))

    # Convert from JSON to R readable format
    edr_data <- jsonlite::fromJSON(txt = rawToChar(x = edr_get$content))

    # If unsuccessful, print message (and try again)
    if(is.null(edr_data$domain)) message(paste0("KNMI EDR API failed to fulfill request with starting date ",
                                                start_date, ". Will try again."))

    # If successful, end
    if(!is.null(edr_data$domain)) break()

  })

  # Output
  return(edr_data)

}



# II. Get temperature data for desired spatiotemporal parameters ---------

# Define bounding box for Veluwe site
bbox <- c(5.824436777442551, 52.032393019069225, 5.870356194968013, 52.046647934312794)

# Retrieve data for 1988 to 2023
temp <- purrr::map(.x = 1988:2023,
                        .f = ~{

                          # Retrieve data for period 1 (1 Dec to 1 March)
                          period1 <- retrieve_knmi_edr_data(bbox = bbox,
                                                            variable = "mean temperature",
                                                            start_date = lubridate::make_date(.x - 1, 12, 1),
                                                            start_time = "00:00:00",
                                                            end_date = lubridate::make_date(.x, 3, 1),
                                                            end_time = "23:59:59")

                          # Retrieve data for period 2 (2 March to 31 May)
                          period2 <- retrieve_knmi_edr_data(bbox = bbox,
                                                            variable = "mean temperature",
                                                            start_date = lubridate::make_date(.x, 3, 2),
                                                            start_time = "00:00:00",
                                                            end_date = lubridate::make_date(.x, 5, 31),
                                                            end_time = "23:59:59")

                          # Match temperature data to dates and coordinates
                          data1 <- tidyr::expand_grid(date = period1$domain$axes$t$values,
                                                      y = period1$domain$axes$y$values,
                                                      x = period1$domain$axes$x$values) |>
                            dplyr::mutate(temperature = period1$ranges$temperature$values)

                          data2 <- tidyr::expand_grid(date = period2$domain$axes$t$values,
                                                      y = period2$domain$axes$y$values,
                                                      x = period2$domain$axes$x$values) |>
                            dplyr::mutate(temperature = period2$ranges$temperature$values)

                          # Bind output of two periods
                          data <- dplyr::bind_rows(data1, data2)

                          return(data)

                        },
                        .progress = TRUE) |>
  purrr::list_c()


# save temperature data
temperature_file <- "/tmp/data/Tg1_seasonalTemperature_Dec1987_to_June2023.csv"
write.csv(temp, temperature_file, row.names = FALSE)

In [ ]:
# Prepare climwin input
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - event_file: String
#    - occurrence_file: String
#    - extendedmeasurementorfact_file: String
#    - budburst_file: String
#    - temperature_file: String
#   outputs:
#    - budburst_climwin_input_file: String
#    - temp_climwin_input_file: String
#   dependencies:
#    - name: dplyr
#    - name: stringr
#    - name: lubridate
#    - name: tidyr
#    - name: geosphere
# ...

# I. Load packages & retrieve data ----------------------------------------------------------

# Read in DwC-A tables
event <- read.csv(event_file)
occ <- read.csv(occurrence_file)
mof <- read.csv(extendedmeasurementorfact_file)

#  Read in processed bud burst data
bud_burst_dates <- read.csv(budburst_file)

# Read in temperature data
temp <- read.csv(temperature_file) %>%
  dplyr::rename("Longitude" = "x",
                "Latitude" = "y")


# II. Prepare data for the climate models -----------------------------------------

## 1. Get relevant information to map trees to temperature grid cells ####
# The temperature data of the KNMI is gridded and for trees to be modeled with their
# respective temperatures, they first need to be matched to the temperature from the grid cell closest to them.

# Get organismID & scientific name from occurrence file and connect measurements with events (i.e., trees)
budburst <- dplyr::right_join(occ %>%
                                dplyr::select("eventID", "organismID", "scientificName"),
                              event, by = "eventID", relationship = "many-to-many") %>%
  dplyr::right_join(bud_burst_dates, by = c("year", "scientificName", "organismID", "verbatimLocality")) %>%
  # filter for Hoge Veluwe
  dplyr::filter(verbatimLocality == "Hoge Veluwe")

# Get the distinct longitude and latitude for temperature
lon_lat_temp <- temp %>%
  dplyr::distinct(Longitude, Latitude)

# Get all individual trees with location information
trees <- budburst %>%
  dplyr::distinct(organismID, .keep_all = TRUE) %>%
  dplyr::filter(!is.na(decimalLongitude))

# Select coordinates of each individual tree
tree_coords <- trees %>%
  dplyr::select("decimalLongitude", "decimalLatitude")


## 2. Map trees to temperature grid cells ####

# Calculate the geographic distance between each tree & each temperature coordinate pair
distance <- as.data.frame(geosphere::distm(tree_coords, lon_lat_temp))

# Find the minimum distance between tree and temperature coordinates
distance$minPos <- apply(distance, 1, which.min)

# Assign position name to coordinate pairs to match with positions of minimum value
lon_lat_temp$Pos <- 1:nrow(lon_lat_temp)

# Assign coordinates of the temperature measures closest to each tree to bud burst data
budburst1 <- dplyr::left_join(distance, lon_lat_temp, by = c("minPos" = "Pos")) %>%
  dplyr::select("tempLon" = "Longitude",
                "tempLat" = "Latitude") %>%
  dplyr::bind_cols(trees, .) %>%
  dplyr::select("organismID", "tempLon", "tempLat") %>%
  dplyr::right_join(budburst, by = "organismID")


# III. Create location IDs for each temperature measure point ------------------

# Assign locID to each temperature-coordinate pair
# Exclude trees without location information
temp_locations <- budburst1 %>%
  dplyr::distinct(tempLon, tempLat) %>%
  tidyr::drop_na() %>%
  dplyr::mutate(locID = paste0("loc", 1:dplyr::n()))

# Add locID to each single temperature measure
temp <-
  temp %>%
  dplyr::left_join(temp_locations, by = c("Latitude" = "tempLat",
                                          "Longitude" = "tempLon"))

# IV. Annual average bud burst dates  ----------------------------

# Assign locID to bud burst measures and get annual average bud burst date per locID
avg_annual_budburst_dates <-
  budburst1 %>%
  dplyr::left_join(temp_locations, by = c("tempLat", "tempLon")) %>%
  dplyr::summarise(avg_bud_burst_DOY = mean(bud_burst_DOY, na.rm = TRUE),
                   .by = c("locID", "year", "scientificName")) %>%
  dplyr::mutate(avg_bud_burst_date = avg_bud_burst_DOY + lubridate::make_date(year, 1, 1) - 1)

# V. Save output files ----------------------------------------------------

# Save bud burst data
budburst_climwin_input_file <- "/tmp/data/budburst_climwin_input.csv"
write.csv(avg_annual_budburst_dates, file = budburst_climwin_input_file,
          row.names = FALSE)

# Save temperature data
temp_climwin_input_file <- "/tmp/data/temp_climwin_input.csv"
write.csv(temp, file = temp_climwin_input_file,
          row.names = FALSE)

In [ ]:
# Find climate window
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - budburst_climwin_input_file: String
#    - temp_climwin_input_file: String
#   dependencies:
#    - name: dplyr
#    - name: stringr
#    - name: lubridate
#    - name: climwin
#   outputs:
#    - firstWindow_file: String
# ...

# I. Retrieve data & load packages ----------------------------------------

# Retrieve temperature data and bud burst data
temp <- read.csv(temp_climwin_input_file)

avg_annual_budburst_dates <- read.csv(budburst_climwin_input_file)


# II. Date conversion to fit climwin  -------------------------------------

# climwin can only handle dates in the format "dd/mm/yyyy" (as a character), so dates
# have to be converted before modelling. However, there need to be numeric dates for the baseline models too.

# Convert dates of temperature file to climwin format
temp <- temp %>%
  dplyr::mutate(date = lubridate::as_date(date),
                year = lubridate::year(date),
                month = lubridate::month(date),
                day = lubridate::day(date),
                doy = lubridate::yday(date),
                # Create dummy for filtering window later. Format: 312 = March 12, 401 = April 1
                dummy = month * 100 + day,
                factor_date = as.factor(paste(day, month, year, sep = "/")))

# Convert bud burst dates
avg_annual_budburst_dates <- avg_annual_budburst_dates %>%
  dplyr::mutate(date_info = paste(year, floor(avg_bud_burst_DOY)),
                date = strptime(date_info, "%Y %j"),
                date = as.factor(format(as.Date(date), "%d/%m/%Y"))) %>%
  # Create numeric dates to be used in the baseline model &
  # and exclude trees without coordinates
  dplyr::mutate(DOY = lubridate::yday(as.Date(avg_bud_burst_date))) |>
  dplyr::filter(!is.na(date), !is.na(locID))




# III. Climwin: Sliding window model --------------------------------------------

## 1. Function to calculate the window in which bud burst data is best explained by temperature ####

# Arguments
# - biological_data: Tibble specifying the biological input data for the climate model, containing biological dates that are tested in format 'dd/mm/yyyy'. Only necessary when the first window is calculated.
# - climate_data: Tibble specifying the climate data that is used as input for the climate model. 
# - reference_day: Numeric vector of 2 values specifying the day and month of the reference day before which climate windows are tested. For example, c(31, 5) for the 31st of March.
# - range: Numeric vector of 2 values specifying the range of days before the reference day in which climate windows are tested. For example, c(181, 0), meaning that windows between 181 days and 0 days before the reference day are tested.
# - window_number: Choice between "first" and "second", specifies whether the first best window should be calculated or a second window based on the first one.
# - first_window: Tibble containing the best model data of the first window/iteration of the function. Used as input data when second window should be calculated.

find_climate_window <- function(biological_data = NULL,
                                climate_data,
                                range,
                                reference_day,
                                window_number = c("first", "second"),
                                first_window = NULL) {

  # Find 'first' or 'second' climate window
  if(window_number == "first") {

    # Return error if biological data is not provided when searching for first window
    if(is.null(biological_data)) {

      stop("If you want to find a first climate window, provide the biological data as `biological_data`.")

    }

    # Define baseline model
    baseline <- lm(DOY ~ year, data = biological_data)

  } else if(window_number == "second") {

    # Return error if first window output is not provided when searching for second window
    if(is.null(first_window)) {

      stop("If you want to find a second climate window, provide the output of the first iteration of `find_climate_window()` as `first_window`.")

    }

    biological_data <- first_window$biological_data

    # The first window is added as an explanatory variable to the baseline model
    baseline_data <- first_window$best_window[[1]]$BestModelData %>%
      dplyr::rename("first_window" = "climate",
                    "DOY" = "yvar")

    # Define baseline model
    baseline <- lm(DOY ~ year + first_window, data = baseline_data)

  }

  # climwin analysis: Find best window
  best_window <- climwin::slidingwin(baseline = baseline,
                                     xvar = list(Temp = climate_data$temperature),
                                     cdate = climate_data$factor_date,
                                     bdate = biological_data$date,
                                     type = "absolute",
                                     refday = reference_day,
                                     spatial = list(biological_data$locID, climate_data$locID),
                                     range = range,
                                     func = "lin",
                                     stat = "mean")

  # Back calculation of the opening and closing day of the calculated window to calender dates

  # Create a reference year for calculation of start and end date
  # Note: can be any year that is not a leap year, as dates should be calculated on the basis of regular years
  reference_year <- dplyr::if_else(condition = lubridate::leap_year(max(climate_data$year)),
                                   true = max(climate_data$year) - 1,
                                   false = max(climate_data$year))

  # Calculate calender date when window opens
  start_date <- lubridate::make_date(year = reference_year,
                                     month = reference_day[2],
                                     day = reference_day[1]) - best_window$combos[1,]$WindowOpen

  # Calculate calender date when window closes
  end_date <- lubridate::make_date(year = reference_year,
                                   month = reference_day[2],
                                   day = reference_day[1]) - best_window$combos[1,]$WindowClose

  return(tibble::lst(best_window, biological_data, baseline, range, reference_day, climate_data, start_date, end_date))

}


## 2. Calculate windows for each species of interest ####
first_window_Qrobur <- find_climate_window(biological_data = avg_annual_budburst_dates %>%
                                             dplyr::filter(stringr::str_detect(scientificName, "Quercus robur")),
                                           climate_data = temp,
                                           window_number = "first",
                                           reference_day = c(31, 5),
                                           range = c(181, 0))

# show start and end date
first_window_Qrobur$start_date
first_window_Qrobur$end_date

# save model output 
firstWindow_file <- "/tmp/data/climwin_outputs_Qrobur.rda"
save(first_window_Qrobur, file = firstWindow_file)

In [ ]:
# Plot climate window output
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - firstWindow_file: String
#   dependencies:
#    - name: dplyr
#    - name: purrr
#    - name: stringr
#    - name: lubridate
#    - name: climwin
#    - name: ggpubr
# ...


# Load model output data 
load(firstWindow_file)

# II. Visualize output of climwin -----------------------------------------

## 1. Function to create two plots for the output of the climwin model ####
# Arguments
# - x: Output list of function "find_climate_window()".

colour_pal <- c("#48d3d3", "#FC8D59", "#D53E4F", "#FFD560", "#3288BD")
options(repr.plot.width = 15, repr.plot.height = 10)
# II. Visualize output of climwin -----------------------------------------

## 1. Function to create two plots for the output of the climwin model ####
# Arguments
# - x: Output list of function "find_climate_window()".

plot_climwin_output <- function(x){

  # create a heat map of model delta AICc values
  AIC_heatmap <- climwin::plotdelta(dataset = x$best_window[[1]]$Dataset,
                                    arrow = TRUE) +
    ggplot2::theme_classic() +
    ggplot2::theme(axis.title.x = element_text(size = 15),
                   axis.title.y = element_text(size = 15),
                   axis.text.x = element_text(size = 15),
                   axis.text.y = element_text(size = 15),
                   title = element_text(size = 16),
                   legend.position = "bottom")

  # get annual mean temperatures of the best window
  mean_temp_in_window <- x$climate_data %>%
    dplyr::filter(dummy > (lubridate::month(x$start_date) * 100 + lubridate::day(x$start_date)) &
                    dummy < (lubridate::month(x$end_date) * 100 + lubridate::day(x$end_date))) %>%
    dplyr::summarise(mean_temp = mean(temperature, na.rm = TRUE),
                     .by = c("year", "locID"))

  # add mean temperatures to annual bud burst data
  annual_budburst_and_temp <- dplyr::left_join(x$biological_data,
                                               mean_temp_in_window %>%
                                                 dplyr::select("year", "mean_temp", "locID"),
                                               by = c("year", "locID"))

  # plot annual mean temperatures of the best window against annual average bud burst dates
  plot_budburst_temperature<-  ggplot2::ggplot(data = annual_budburst_and_temp,
                                               mapping = aes(x = mean_temp, y = avg_bud_burst_DOY, colour = locID)) +
    ggplot2::geom_point(size = 2, alpha = 0.4) +
    ggplot2::geom_smooth(method = "lm", formula = y ~ x, se = FALSE) +
    ggplot2::theme_classic() +
    ggplot2::scale_colour_manual(values = colour_pal) +
    ggplot2::labs(title = "Bud burst date ~ mean temperature in best window",
                  x = "Annual mean temperature [°C]",
                  y = "Annual mean bud burst date",
                  colour = "Location (grid cell)") +
    ggplot2::theme(title = element_text(size = 16),
                   axis.title.x = element_text(size = 15),
                   axis.title.y = element_text(size = 15),
                   legend.title = element_text(size = 15),
                   axis.text.x = element_text(size = 15),
                   axis.text.y = element_text(size = 15),
                   legend.text = element_text(size = 13),
                   legend.position = "bottom")

  # arrange both plots in one figure
  ggpubr::ggarrange(AIC_heatmap,  plot_budburst_temperature, align = "hv")

}

## 2. Call function and create plots ####

Fig_Qrobur <- plot_climwin_output(first_window_Qrobur)
Fig_Qrobur

In [ ]:
# Retrieve scenario data
# ---
# NaaVRE:
#  cell:
#   dependencies:
#    - name: dplyr
#    - name: purrr
#    - name: stringr
#    - name: httr
#    - name: readr
#    - name: lubridate
#    - name: tidyr
#   outputs:
#    - scenario_data_all_file: String
# ...

dir.create("/tmp/data")

# get master branch of repository where scenario data is strored in
master_branch <- httr::GET("https://api.github.com/repos/matt-long/bio-pop-ToE/git/trees/master?recursive=1")

# get the file paths of each file that needs to be downloaded
file_path <- tibble::tibble(path = purrr::map_chr(httr::content(master_branch)$tree, "path")) %>%
  tidyr::separate_wider_delim(path, delim = '/', names = c('base', 'folder', 'filename'),
                              too_few = "align_end", too_many = "drop") %>%
  dplyr::filter(folder == 'data', stringr::str_detect(filename, '.csv'))

# retrieve data
scenario_data_all <- tibble::tibble()

for (i in seq(1, nrow(file_path))){

  # get path name for each file
  path <- paste0('https://raw.githubusercontent.com/matt-long/bio-pop-ToE/master/notebooks/data/', file_path$filename[i])

  # read each file as CSV
  df <- readr::read_csv(httr::content(httr::GET(path)))

  # assign file name to data
  file_name <- file_path$filename[i]

  # extract scenario name from file name
  df1 <- df %>%
    dplyr::mutate(scenario_name = stringr::str_extract(file_name, pattern = "1pt5degC(?=\\.)|1pt5degC_OS|2pt0degC|RCP85|RCP45"),
                  member_id = as.character(member_id))

  scenario_data_all <- bind_rows(scenario_data_all, df1)

}

# discard data before 1985-01-01 to limit file size
scenario_data_all <- scenario_data_all %>%
  dplyr::filter(time >= as.POSIXct("1985-01-01", tz = "UTC"))

# save scenario data
scenario_data_all_file <- "/tmp/data/scenario_temperatures.csv"
write.csv(scenario_data_all, file = scenario_data_all_file, row.names = FALSE)


In [ ]:
# Validate forecasting model
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - budburst_climwin_input_file: String
#    - temp_climwin_input_file: String
#    - firstWindow_file: String
#    - scenario_data_all_file: String
#   dependencies:
#    - name: dplyr
#    - name: purrr
#    - name: stringr
#    - name: lubridate
#    - name: tidyr
#    - name: ggpubr
#   outputs:
#    - validation_all_zScores_file: String
# ...

# load data
temp <- read.csv(temp_climwin_input_file)
avg_annual_budburst_dates <- read.csv(budburst_climwin_input_file)
climwin_QRobur <- load(firstWindow_file)
scenario_data_all <- read.csv(scenario_data_all_file)

# drop run member_id "009" and "010" from scenario 1pt5degC
scenario_data_all <- scenario_data_all %>%
  dplyr::filter(!(scenario_name %in% c("1pt5degC") & member_id %in% c("006", "007", "009", "010")))


# set seed
set.seed(2804)

# set colour palette
scenario_colours <- c("measured" = "#D53E4F", "RCP45" = "#B9A6E2" , "1pt5degC_OS" = "#FFD560",
                      "2pt0degC" = "#48d3d3", "RCP85" = "#FC8D59", "1pt5degC" = "#3288BD")


# II. Function 1: Format measured temperature data & model bud burst ------

# Arguments
# measured_temperatures: data frame/tibble specifying the measured temperatures that were used as an input for the climate window analysis
# climwin_output: output from the climate window analysis of climwin
# biological_data: data frame/tibble specifying the biological input data (as used for the climate window analysis)
# use_zScores: yes or no, specifying whether the zScores of temperatures should be used to model the biological variable or the actual yearly mean temperatures
# number_simulations: numeric specifying the number of times the prediction of the biological variable is repeated
# scenario_data: data frame containing the scenario temperature data
# scenario: character, specifiying for which scenario the function is run, options are: 1pt5degC, 1pt5degC_OS, 2pt0degC, RCP85, RCP45

model_validation <- function(measured_temperatures,
                             climwin_output,
                             biological_data,
                             use_zScores = c("yes", "no"),
                             number_simulations,
                             scenario_data,
                             scenario = c("1pt5degC, 1pt5degC_OS, 2pt0degC, RCP85, RCP45")) {

  ## 1. Measured temperatures: Formatting ####

  ## convert dates and filter for bud burst sensitive period
  measured_temp <- measured_temperatures %>%
    dplyr::mutate(date = lubridate::as_date(date),
                  year = lubridate::year(date),
                  month = lubridate::month(date),
                  day = lubridate::day(date),
                  doy = lubridate::yday(date),
                  # Create dummy for filtering window later. Format: 312 = March 12, 401 = April 1
                  dummy = month * 100 + day) %>%
    ### filter for the calculated window of climwin
    dplyr::filter(dummy > (lubridate::month(climwin_output$start_date) * 100 + lubridate::day(climwin_output$start_date)) &
                    dummy < (lubridate::month(climwin_output$end_date) * 100 + lubridate::day(climwin_output$end_date))) %>%
    ### get mean temperature per day and year over all locations
    dplyr::summarise("mean_temperature" = mean(temperature),
                     "sd_KNMI_temp" = sd(temperature, na.rm = TRUE),
                     .by = "year") %>%
    dplyr::mutate(overall_mean = mean(mean_temperature),
                  overall_sd = sd(mean_temperature),
                  # annotate data
                  type = "measured",
                  scenario = NA,
                  run = NA,
                  # z-scores as (mean of year x - mean over years)/sd of yearly means
                  zScore = (mean_temperature - overall_mean) / overall_sd) %>%
    dplyr::select(!c("overall_mean", "overall_sd"))



  ## 2. Scenario temperatures: Formatting ####
  df <- scenario_data %>%
    dplyr::filter(scenario_name == scenario)

  scenario_temp_fut <- NULL
  scenario_temp_hist <- NULL

  for (a in unique(df$member_id)) {

    ## add year and day of year
    scenario_temp <- df %>%
      dplyr::filter(member_id == a) %>%
      dplyr::mutate(date = lubridate::as_date(time),
                    year = lubridate::year(date),
                    dummy = lubridate::month(date) * 100 + lubridate::day(date),
                    doy = lubridate::yday(date),
                    # convert mean temperatures from Kelvin to degree Celsius
                    temp_degreesC = as.numeric(TREFHT) - 273.15) %>%
      dplyr::filter(dummy > (lubridate::month(climwin_output$start_date) * 100 + lubridate::day(climwin_output$start_date)) &
                      dummy < (lubridate::month(climwin_output$end_date) * 100 + lubridate::day(climwin_output$end_date))) %>%

      ## summarise and annotate data per run
      dplyr::mutate("mean_temperature" = mean(temp_degreesC, na.rm = TRUE),
                    "sd_temperature" = sd(temp_degreesC, na.rm = TRUE),
                    .by = "year") %>%
      dplyr::mutate(type = "model",
                    run = a)

    ## standardize scenario temperatures for historic and future period
    ### historic period
    scenario_temp_hist_perRun <- scenario_temp %>%
      dplyr::filter(year >= min(biological_data$year), year <= max(biological_data$year)) %>%
      # get mean and standard deviation over all years
      dplyr::mutate(overall_mean_hist = mean(mean_temperature),
                    overall_sd_hist = sd(mean_temperature),
                    # z-scores as (mean of year x - mean over years)/sd of yearly means
                    zScore = (mean_temperature - overall_mean_hist) / overall_sd_hist)

    overall_mean_hist <- unique(scenario_temp_hist_perRun$overall_mean_hist)

    scenario_temp_hist <- rbind(scenario_temp_hist, scenario_temp_hist_perRun)

    ### future period
    scenario_temp_fut_perRun <- scenario_temp %>%
      dplyr::filter(year > max(biological_data$year)) %>%
      # get mean and standard deviation over all years
      dplyr::mutate(overall_mean_fut = mean(mean_temperature),
                    overall_sd_fut = sd(mean_temperature),
                    # z-scores as (mean of year x - mean over years)/sd of yearly means
                    zScore = (mean_temperature - overall_mean_hist) / overall_sd_fut)

    scenario_temp_fut <- rbind(scenario_temp_fut, scenario_temp_fut_perRun)

  }

  ## 3. Biological data ####

  ## get annual mean bud burst date
  avg_budburst <- biological_data  %>%
    dplyr::mutate(year = as.numeric(year)) %>%
    dplyr::summarise(avg_budburst_DOY_allLoc = mean(avg_bud_burst_DOY, na.rm = TRUE),
                     .by = "year")

  ## add bud burst to temperature
  budburst_temp <- measured_temp %>%
    dplyr::left_join(avg_budburst, by = "year")


  ## 4. Modelling ####

  # get slope of observed biological data ~ year
  m1 <- lm(avg_budburst_DOY_allLoc ~ year, data = budburst_temp)
  slope_bb_year <- m1$coefficients[2]

  ## 4.1 Modelling with mean temperatures ####

  if(!(use_zScores == "yes")) {

    ### model observed bud burst with mean measured temperatures
    model_for_prediction <- lm(avg_budburst_DOY_allLoc ~ mean_temperature, data = budburst_temp)

    ### get model parameters
    intercept_bb_temp <- as.numeric(model_for_prediction$coefficients[1])
    slope_bb_temp <- as.numeric(model_for_prediction$coefficients[2])
    sigma_bb_temp <- sigma(model_for_prediction)
    model_coefficients_bb_temp <- model_for_prediction$coefficients
    vcov_bb_temp <- vcov(model_for_prediction)

    # Simulate prediction of biological data based on model_for_prediction for measured temperatures
    sim_measured_slope_pred_year <- NULL

    for (s in 1:number_simulations) {

      # get data to predict on
      new_data <- data.frame(mean_temperature = budburst_temp$mean_temperature)

      residual_error <- rnorm(n = nrow(new_data), sd = sigma_bb_temp)
      predicted_bb_date <- intercept_bb_temp + slope_bb_temp * new_data$mean_temperature + residual_error
      predicted_budburst <- data.frame(budburst_temp, predicted_bb_date)

      # get slope
      model_pred_year <- lm(predicted_bb_date ~ year, data = predicted_budburst)
      slope_pred_year <- as.numeric(model_pred_year$coefficients[2])
      df_slope_pred_year <- data.frame(scenario = "measured", run = NA, sim = paste0("sim_", s), slope = slope_pred_year)

      sim_measured_slope_pred_year <- rbind(sim_measured_slope_pred_year, df_slope_pred_year)

    }

    # Simulate predictions of biological data based on model_for_prediction for scenario temperatures
    sim_scenario_slope_pred_year <- NULL

    for (s in 1:number_simulations) {

      scenario_slopes_pred_year <- NULL

      for (r in unique(scenario_temp_hist$run)) {

        # filter per run
        df <- scenario_temp_hist %>%
          dplyr::filter(run == r)

        # get data to predict on
        new_data <- data.frame(mean_temperature = df$mean_temperature)

        residual_error <- rnorm(n = nrow(new_data), sd = sigma_bb_temp)
        predicted_bb_date <- intercept_bb_temp + slope_bb_temp * new_data$mean_temperature + residual_error
        predicted_budburst <- data.frame(df, predicted_bb_date)

        # get slope
        model_pred_year <- lm(predicted_bb_date ~ year, data = predicted_budburst)
        slope_pred_year <- as.numeric(model_pred_year$coefficients[2])
        df_slope_pred_year <- data.frame(scenario = scenario, run = r, sim = paste0("sim_", 1), slope = slope_pred_year)

        # add data to data frames
        scenario_slopes_pred_year <- rbind(scenario_slopes_pred_year, df_slope_pred_year)
      }

      sim_scenario_slope_pred_year <- rbind(sim_scenario_slope_pred_year, scenario_slopes_pred_year)
    }

    # combine simulated slopes for measured temperatures and scenario temperatures
    slopes_combined <- rbind(sim_measured_slope_pred_year, sim_scenario_slope_pred_year)

    # visually compare slopes
    plot_validation <- ggplot2::ggplot(data = slopes_combined) +
      ggplot2::geom_histogram(mapping = ggplot2::aes(y = ggplot2::after_stat(density),
                                                     x = slope,
                                                     fill = scenario),
                              colour = "black",
                              alpha = 0.7,
                              position = "identity",
                              binwidth = 0.01) +
      ggplot2::scale_fill_manual(values = scenario_colours) +
      ggplot2::geom_vline(xintercept = slope_bb_year,
                          linewidth = 2) +
      ggplot2::theme_classic() +
      ggplot2::labs(x = "Slope (predicted bud burst ~ year)", y = "Density")

    ### plot observed bud burst dates against mean temperatures
    plot_obs_temp <-  ggplot2::ggplot() +
      ggplot2::geom_abline(intercept = intercept_bb_temp,
                           slope = slope_bb_temp,
                           linetype = 1,
                           linewidth = 1,
                           color = "black") +
      ggplot2::geom_point(data = budburst_temp,
                          mapping = ggplot2::aes(x = mean_temperature,
                                                 y = avg_budburst_DOY_allLoc),
                          color = "black", size = 3) +
      ggplot2::ylab("Bud burst date (DOY)") +
      ggplot2::xlab("mean measured temperature") +
      ggplot2::theme_classic()

    # TODO: Add a later stage, match these output names to the ones when 'use_zScores == "yes"'
    return(tibble::lst(budburst_temp,
                       scenario_temp_hist,
                       scenario_temp_fut,
                       model_for_prediction,
                       plot_validation,
                       plot_obs_temp))

  }

  if(use_zScores == "yes") {

    ## 4.2. Modelling with zScores of temperature ####

    ### model observed bud burst with zScores of measured temperatures
    model_for_prediction_zScore <- lm(avg_budburst_DOY_allLoc ~ zScore, data = budburst_temp)

    ### get model parameter
    intercept_bb_temp_zScore <- as.numeric(model_for_prediction_zScore$coefficients[1])
    slope_bb_temp_zScore <- as.numeric(model_for_prediction_zScore$coefficients[2])
    sigma_bb_temp_zScore <- stats::sigma(model_for_prediction_zScore)
    model_coefficients_bb_temp_zScore <- model_for_prediction_zScore$coefficients
    vcov_bb_temp_zScore <- stats::vcov(model_for_prediction_zScore)

    # Simulate prediction of biological data based on model_for_prediction for measured temperatures
    sim_measured_slope_pred_year <- NULL

    for (s in 1:number_simulations) {

      # get data to predict on
      new_data <- data.frame(zScore = budburst_temp$zScore)

      residual_error <- rnorm(n = nrow(new_data), sd = sigma_bb_temp_zScore)
      predicted_bb_date <- intercept_bb_temp_zScore + slope_bb_temp_zScore * new_data$zScore + residual_error
      predicted_budburst <- data.frame(budburst_temp, predicted_bb_date)

      # get slope
      model_pred_year <- lm(predicted_bb_date ~ year, data = predicted_budburst)
      slope_pred_year <- as.numeric(model_pred_year$coefficients[2])
      df_slope_pred_year <- data.frame(scenario = "measured", run = NA, sim = paste0("sim_", s), slope = slope_pred_year)


      sim_measured_slope_pred_year <- rbind(sim_measured_slope_pred_year, df_slope_pred_year)

    }

    # Simulate predictions of biological data based on model_for_prediction for scenario temperatures
    sim_scenario_slope_pred_year <- NULL

    for (s in 1:number_simulations) {

      scenario_slopes_pred_year <- NULL

      for (r in unique(scenario_temp_hist$run)) {

        # filter per run
        df <- scenario_temp_hist %>%
          dplyr::filter(run == r)

        # get data to predict on
        new_data <- data.frame(zScore = df$zScore)

        residual_error <- rnorm(n = nrow(new_data), sd = sigma_bb_temp_zScore)
        predicted_bb_date <- intercept_bb_temp_zScore + slope_bb_temp_zScore * new_data$zScore + residual_error
        predicted_budburst <- data.frame(df, predicted_bb_date)

        # get slope
        model_pred_year <- lm(predicted_bb_date ~ year, data = predicted_budburst)
        slope_pred_year <- as.numeric(model_pred_year$coefficients[2])
        df_slope_pred_year <- data.frame(scenario = scenario, run = r, sim = paste0("sim_", s), slope = slope_pred_year)

        # add data to data frames
        scenario_slopes_pred_year <- rbind(scenario_slopes_pred_year, df_slope_pred_year)

      }

      sim_scenario_slope_pred_year <- rbind(sim_scenario_slope_pred_year, scenario_slopes_pred_year)

    }

    # combine simulated slopes for measured temperatures and scenario temperatures
    slopes_combined <- rbind(sim_measured_slope_pred_year, sim_scenario_slope_pred_year)

    # visually compare slopes
    plot_validation <- ggplot2::ggplot(data = slopes_combined) +
      ggplot2::geom_histogram(mapping = ggplot2::aes(y = ggplot2::after_stat(density),
                                                     x = slope,
                                                     fill = scenario),
                              colour = "black",
                              alpha = 0.7,
                              position = "identity",
                              binwidth = 0.01) +
      ggplot2::scale_fill_manual(values = scenario_colours) +
      ggplot2::geom_vline(xintercept = slope_bb_year, linewidth = 2) +
      ggplot2::theme_classic() +
      ggplot2::labs(x = "Slope (predicted bud burst ~ year)", y = "Density")


    ### plot observed bud burst dates against zScores of temperature
    plot_zScore <-  ggplot2::ggplot() +
      ggplot2::geom_abline(intercept = intercept_bb_temp_zScore,
                           slope = slope_bb_temp_zScore,
                           linetype = 1,
                           linewidth = 1,
                           color = "black") +
      ggplot2::geom_point(data = budburst_temp,
                          mapping = ggplot2::aes(x = zScore,
                                                 y = avg_budburst_DOY_allLoc),
                          color = "black",
                          size = 3) +
      ggplot2::ylab("Bud burst date (DOY)") +
      ggplot2::xlab("zScore measured temperatures") +
      ggplot2::theme_classic()

    return(tibble::lst(budburst_temp,
                       scenario_temp_hist,
                       scenario_temp_fut,
                       model_for_prediction_zScore,
                       plot_validation,
                       plot_zScore))

  }

}

# IV. Run function for all scenarios in one go -----------------------------

validation_all_zScores <- purrr::map(.x = c("1pt5degC", "1pt5degC_OS", "2pt0degC", "RCP85", "RCP45") %>%
                                       purrr::set_names(),
                                     .f = ~{

                                       output <- model_validation(measured_temperatures = temp,
                                                                  climwin_output = first_window_Qrobur,
                                                                  biological_data = avg_annual_budburst_dates %>%
                                                                    dplyr::filter(stringr::str_detect(scientificName, "Quercus robur")),
                                                                  scenario = .x,
                                                                  scenario_data = scenario_data_all,
                                                                  use_zScores = "yes",
                                                                  number_simulations = 100)

                                       return(output)

                                     },
                                     .progress = TRUE)

# save validation output file 
validation_all_zScores_file <- "/tmp/data/validation_all_zScores.Rda"
save(validation_all_zScores, file = validation_all_zScores_file)
 
validation_plot_all <- ggpubr::ggarrange(plotlist = purrr::map(.x = validation_all_zScores, "plot_validation"),
                                         nrow = 3, ncol = 2)

validation_plot_all

In [ ]:
# Forecast bud burst
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - validation_all_zScores_file: String
#   dependencies:
#    - name: dplyr
#    - name: purrr
#    - name: stringr
#    - name: here
#    - name: tidyr
# ...

# load input 
load(validation_all_zScores_file)

# set colour palette
scenario_colours <- c("measured" = "#D53E4F", "RCP45" = "#B9A6E2" , "1pt5degC_OS" = "#FFD560",
                      "2pt0degC" = "#48d3d3", "RCP85" = "#FC8D59", "1pt5degC" = "#3288BD")

# II. Function: Forecasting ---------------------------------------------------------

# Arguments
# scenario_temp: Formatted scenario temperature data as given by in the output list of function "scenario_hindcast_budburst".
# linear_model: Linear model used in function 'model_budburst_measuredTemp' to model bud burst against the measured temperatures. Used as the basis for the predictions of future bud burst dates.
# use_zScores: yes or no, specifying whether the zScores of temperatures should be used to model the biological variable or the actual yearly mean temperatures

forecasting_budburst <- function(scenario_temp,
                                 linear_model,
                                 use_zScores = c("yes", "no")){

  if(use_zScores == "yes"){

    model_coefficients_bb_temp_zScore <- linear_model$coefficients
    vcov_bb_temp_zScore <- stats::vcov(linear_model)

    forecasted_budburst_perRun <- NULL
    forecasted_budburst_smoothend <- NULL

    for (r in unique(scenario_temp$run)) {

      df <- scenario_temp %>%
        dplyr::filter(run == r)

      X <- stats::model.matrix(~ zScore, data = df)
      predicted <- X %*% model_coefficients_bb_temp_zScore
      varPred <- diag(X %*% vcov_bb_temp_zScore %*% t(X))
      sePred <- sqrt(varPred)

      df1 <- data.frame(df,
                        predicted_bb_date = predicted,
                        varPred = varPred,
                        sePred = sePred)

      df1$Con95Pred <- 1.96 * df1$sePred

      forecasted_budburst_perRun <- rbind(forecasted_budburst_perRun, df1)


      # 11 year sliding-window to smooth forecasting
      df2 <- df1 %>% dplyr::distinct(year, .keep_all = TRUE)

      forecasted_budburst_smoothend_window <- NULL

      for (s in (min(df2$year) + 5) : (max(df2$year) - 5)) {

        Data_s <- df2 %>% dplyr::filter(year >= (s - 5) & year <= (s + 5))

        Data_new <- data.frame(year = s,
                               scenario_name = unique(df2$scenario_name),
                               run = r,
                               mean_pred_bb_window = mean(Data_s$predicted_bb_date),
                               sd = sd(Data_s$predicted_bb_date),
                               error = (qnorm(0.95) * sd(Data_s$predicted_bb_date))/sqrt(nrow(Data_s)))

        Data_new <- Data_new[stats::complete.cases(Data_new), ]

        forecasted_budburst_smoothend_window <- rbind(forecasted_budburst_smoothend_window, Data_new)
      }

      forecasted_budburst_smoothend <- rbind(forecasted_budburst_smoothend, forecasted_budburst_smoothend_window)
  }

    return(tibble::lst(forecasted_budburst_perRun, forecasted_budburst_smoothend))

  } else {

    # use mean temperatures

    model_coefficients_bb_temp <- linear_model$coefficients
    vcov_bb_temp <- stats::vcov(linear_model)

    forecasted_budburst_perRun <- NULL
    forecasted_budburst_smoothend <- NULL

    for (r in unique(scenario_temp$run)) {

      df <- scenario_temp %>%
        dplyr::filter(run == r)

      X <- stats::model.matrix(~ mean_temperature, data = df)
      predicted <- X %*% model_coefficients_bb_temp
      varPred <- diag(X %*% vcov_bb_temp %*% t(X))
      sePred <- sqrt(varPred)

      df1 <- data.frame(df,
                        predicted_bb_date = predicted,
                        varPred = varPred,
                        sePred = sePred)

      df1$Con95Pred <- 1.96 * df1$sePred

      forecasted_budburst_perRun <- rbind(forecasted_budburst_perRun, df1)


      # 11 year sliding-window to smooth forecasting
      df2 <- df1 %>% dplyr::distinct(year, .keep_all = TRUE)

      forecasted_budburst_smoothend_window <- NULL

      for (s in (min(df2$year) + 5) : (max(df2$year) - 5)) {

        Data_s <- df2 %>% dplyr::filter(year >= (s - 5) & year <= (s + 5))

        Data_new <- data.frame(year = s,
                               scenario_name = unique(df2$scenario_name),
                               run = r,
                               mean_pred_bb_window = mean(Data_s$predicted_bb_date),
                               sd = sd(Data_s$predicted_bb_date),
                               error = (qnorm(0.95) * sd(Data_s$predicted_bb_date))/sqrt(nrow(Data_s)))

        Data_new <- Data_new[stats::complete.cases(Data_new), ]

        forecasted_budburst_smoothend_window <- rbind(forecasted_budburst_smoothend_window, Data_new)
      }

      forecasted_budburst_smoothend <- rbind(forecasted_budburst_smoothend, forecasted_budburst_smoothend_window)
    }

    return(tibble::lst(forecasted_budburst_perRun, forecasted_budburst_smoothend))

  }

}


# III. Forecast bud burst dates -------------------------------------------

# use zScores
# All scenarios
future_budburst_zScores <- purrr::map(.x = validation_all_zScores,
                                      .f = ~{

                                        output <- forecasting_budburst(scenario_temp = .x$scenario_temp_fut,
                                                                       linear_model = .x$model_for_prediction,
                                                                       use_zScores = "yes")

                                        return(output)

                                      },
                                      .progress = TRUE)

forecasting_all <- purrr::map(.x = future_budburst_zScores,
                              "forecasted_budburst_smoothend") |>
  dplyr::bind_rows()

# plot predicted bud burst of each scenario
forecasting_plot <- forecasting_all %>%
  dplyr::group_by(year, scenario_name) %>%
  dplyr::summarise(mean = mean(mean_pred_bb_window, na.rm = TRUE),
                   sd = sd(mean_pred_bb_window, na.rm = TRUE),
                   error = (qnorm(0.95) * sd)/sqrt(n()),
                   CI_lower = mean - error,
                   CI_upper = mean + error) %>%
  ggplot2::ggplot() +
  ggplot2::geom_line(mapping = ggplot2::aes(y = mean,
                                            x = year,
                                            colour = scenario_name),
                     linewidth = 2) +
  ggplot2::geom_ribbon(mapping = ggplot2::aes(x = year,
                                              ymin = CI_lower,
                                              ymax = CI_upper,
                                              fill = scenario_name),
                       alpha = 0.1) +
  ggplot2::scale_color_manual(values = scenario_colours) +
  ggplot2::scale_fill_manual(values = scenario_colours) +
  ggplot2::theme_classic() +
  ggplot2::labs(x = "Year",
                y = "Predicted bud burst date (mean over scenario runs)",
                title = "Predicted bud burst dates of the future",
                colour = "Scenario",
                fill = "Scenario") +
  ggplot2::theme(legend.position = "bottom")

forecasting_plot